In [1]:
import qiskit
import numpy as np

from PatchedMeasCal.tensor_patch_cal import TensorPatchFitter
from PatchedMeasCal.inv_measure_methods import aim, sim
from PatchedMeasCal.jigsaw import jigsaw
from PatchedMeasCal.qiskit_meas_fitters import qiskit_full, qiskit_linear


from PatchedMeasCal.fake_backends import LocalSimulator


from PatchedMeasCal import state_prep_circuits
from PatchedMeasCal.fake_measurement_distributions import renormalise_measurement_results
from PatchedMeasCal.utils import Progressbar

from PatchedMeasCal.fake_backends import Grid, Hexagonal16, FullyConnected

from PatchedMeasCal.state_prep_circuits import GHZ_prep, GHZ_state_dist
from PatchedMeasCal.bv import bv_circuit_cmap



from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
from qiskit.ignis.mitigation.measurement import tensored_meas_cal, TensoredMeasFitter

from functools import partial

import qiskit.tools.jupyter

import mthree

import random

qiskit.IBMQ.load_account()
provider = qiskit.IBMQ.get_provider()

%qiskit_job_watcher

Accordion(children=(VBox(layout=Layout(max_width='710px', min_width='710px')),), layout=Layout(max_height='500…

<IPython.core.display.Javascript object>

In [2]:
def distance_measure(x, bv_value):
    if bv_value in x:
        return 1 - (x[bv_value] / sum(x.values()))
    return 1

## Setup
The idea here is that each approachs gets a maximum of 32000 measurement shots to spend
- Backend style methods will spend 50% of that on the build, 50% of that on the run
- Circuit style methods will spend 50% on their full run, and the other 50% on other circuits that need to be exectued
- AIM will divide theirs up evenly

In [3]:
n_qubits = 7
n_meas_shots = 16000
n_build_shots = 16000
#n_shots_qiskit_full = n_build_shots // (2 ** n_qubits) 
#n_shots_qiskit_partial = n_build_shots // (n_qubits)


n_circuit_method_shots = n_meas_shots + n_build_shots
backend = provider.get_backend('ibm_nairobi')

circuit = GHZ_prep(backend)

n_shots_cmc = n_build_shots // (2 * len(backend.configuration().coupling_map))

err_cmap = [[0, 6], [3, 6], [4, 6], [0, 2], [1, 2], [2, 5], [2, 6]]

## Mitigation Methods Build

In [4]:
#full_filter = qiskit_full(backend, n_qubits, n_shots_qiskit_full)

# n Circuits to execute
#linear_filter = qiskit_linear(backend, n_qubits, n_shots_qiskit_partial)

mit = mthree.M3Mitigation(backend)
mit.cals_from_system(list(range(n_qubits)), n_build_shots // 2)

tpf_err = TensorPatchFitter(backend, n_shots=n_shots_cmc, coupling_map=err_cmap)
tpf_err.build(verbose=True)

tpf = TensorPatchFitter(backend, n_shots=n_shots_cmc)
tpf.build(verbose=True)

ERROR:websocket:close status: 4002


Building Coupling Graph
Building Edge Calibrations
	Building Calibration Circuits
	Building Edge Patches : [====================] 100.0%  ETA: 0s
	Building Edge Patches : [====================] 100.0%  ETA: 0s
	Constructing Calibration Circuits : [====================] 100.0%  ETA: 0s
	Executing Calibration Circuits


ERROR:websocket:close status: 4002


	De-hexing Measurement Results
	Tracing Patched Calibration Results : [====================] 100.0%  ETA: 0s
Building Patch Calibrations
	Merging Patches : [====================] 100.0%  ETA: 0s
Building Measure Fitter
	Building Meas Fitters from Patches : [====================] 100.0%  ETA: 0s
Building Coupling Graph
Building Edge Calibrations
	Building Calibration Circuits
	Building Edge Patches : [====================] 100.0%  ETA: 0s
	Building Edge Patches : [====================] 100.0%  ETA: 0s
	Constructing Calibration Circuits : [====================] 100.0%  ETA: 0s
	Executing Calibration Circuits


ERROR:websocket:close status: 4002


	De-hexing Measurement Results
	Tracing Patched Calibration Results : [====================] 100.0%  ETA: 0s
Building Patch Calibrations
	Merging Patches : [====================] 100.0%  ETA: 0s
Building Measure Fitter
	Building Meas Fitters from Patches : [====================] 100.0%  ETA: 0s


## Run with repetitions
If everything above is working, let's crank it out a few times

In [5]:
n_reps = 10
results = {
    'bare':[],
    'full':[],
    'linear':[],
    'aim':[],
    'sim':[],
    'jigsaw':[],
    'cmc':[],
    'cmc_err':[],
    'm3':[]
}

for _ in range(n_reps):

    target_value = random.randint(0, 2 ** (n_qubits - 1) - 1)
    bv_value = bin((target_value << 1) + 1)[2:].zfill(n_qubits)[::-1]
    bv_string = bin(target_value)[2:].zfill(n_qubits - 1)
    circuit = bv_circuit_cmap(bv_string, n_qubits, backend)
    dist_measure = lambda x: distance_measure(x, bv_value)
    
    bare_result_job = qiskit.execute(circuit, 
                         backend, 
                         shots=n_meas_shots, 
                         optimization_level=0,
                         initial_layout=list(range(n_qubits))
                        )
    
    
    bare_result = bare_result_job.result().get_counts()
    
    results['bare'].append(
        dist_measure(bare_result)
    )
#     results['full'].append(
#         dist_measure(full_filter.apply(bare_result))
#     )
#     results['linear'].append(
#         dist_measure(linear_filter.apply(bare_result))
#     )
    results['cmc'].append(
        dist_measure(tpf.apply(bare_result))
    )
    results['cmc_err'].append(
        dist_measure(tpf_err.apply(bare_result))
    )
    
    results ['m3'].append(
        dist_measure(mit.apply_correction(bare_result, list(range(n_qubits))).nearest_probability_distribution())
    )
    
    results['aim'].append(dist_measure(aim(circuit, backend, n_qubits, n_shots=n_circuit_method_shots, equal_shot_distribution=True)))
    results['sim'].append(dist_measure(sim(circuit, backend, n_qubits, n_shots=n_circuit_method_shots, equal_shot_distribution=True)))
    results['jigsaw'].append(dist_measure(jigsaw(circuit, backend, n_circuit_method_shots, equal_shot_distribution=True)))
    

ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:error from callback <bound method BaseWebsocketClient.on_close of <qiskit.providers.ibmq.api.clients.websocket.WebsocketClient object at 0x7fe729da0f40>>: on_close() missing 2 required positional arguments: 'status_code' and 'msg'
ERROR:websocket:error from callback <bound method BaseWebsocketClient.on_close of <qiskit.providers.ibmq.api.clients.websocket.WebsocketClient object at 0x7fe729da0f40>>: on_close() missing 2 required positional arguments: 'status_code' and 'msg'
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:error from callback <bound method B

In [6]:
results

{'bare': [0.7208749999999999,
  0.5760624999999999,
  0.6710625,
  0.724375,
  0.864375,
  0.684875,
  0.44243750000000004,
  0.803875,
  0.46843749999999995,
  0.76175],
 'full': [],
 'linear': [],
 'aim': [0.7422500000000001,
  0.47058333333333335,
  0.6787916666666667,
  0.6119583333333334,
  0.6756249999999999,
  0.6472916666666666,
  0.3121666666666667,
  0.6566666666666667,
  0.5066666666666666,
  0.632],
 'sim': [0.7308749999999999,
  0.5041875,
  0.7306250000000001,
  0.59309375,
  0.70521875,
  0.6349687500000001,
  0.457125,
  0.6676875,
  0.467125,
  0.65478125],
 'jigsaw': [0.5285125262688721,
  0.3916630651855437,
  0.40871791410210867,
  0.3945949381972672,
  0.776821158031914,
  0.47608816953204636,
  0.12166453948783107,
  0.5951038102006412,
  0.11142074497968446,
  0.680977998454185],
 'cmc': [0.6051818802488758,
  0.41490058235337346,
  0.5322106280037229,
  0.5988334096426489,
  0.7823496567861561,
  0.5243299393624945,
  0.286550839639391,
  0.7069905754028364,
  0

In [16]:
bv_nairobi_results = {'bare': [0.7208749999999999,
  0.5760624999999999,
  0.6710625,
  0.724375,
  0.864375,
  0.684875,
  0.44243750000000004,
  0.803875,
  0.46843749999999995,
  0.76175],
 'aim': [0.7422500000000001,
  0.47058333333333335,
  0.6787916666666667,
  0.6119583333333334,
  0.6756249999999999,
  0.6472916666666666,
  0.3121666666666667,
  0.6566666666666667,
  0.5066666666666666,
  0.632],
 'sim': [0.7308749999999999,
  0.5041875,
  0.7306250000000001,
  0.59309375,
  0.70521875,
  0.6349687500000001,
  0.457125,
  0.6676875,
  0.467125,
  0.65478125],
 'jigsaw': [0.5285125262688721,
  0.3916630651855437,
  0.40871791410210867,
  0.3945949381972672,
  0.776821158031914,
  0.47608816953204636,
  0.12166453948783107,
  0.5951038102006412,
  0.11142074497968446,
  0.680977998454185],
 'cmc': [0.6051818802488758,
  0.41490058235337346,
  0.5322106280037229,
  0.5988334096426489,
  0.7823496567861561,
  0.5243299393624945,
  0.286550839639391,
  0.7069905754028364,
  0.32077421550443574,
  0.6519572988426896],
 'cmc_err': [0.648227806036496,
  0.4543836801886403,
  0.5490573610967993,
  0.636294301904913,
  0.7927625754386634,
  0.6050892955004313,
  0.32179022202263674,
  0.7455639699504425,
  0.33280798924586175,
  0.6937318925494218],
 'm3': [0.623299720381186,
  0.21100765495975715,
  0.5696476258456218,
  0.4712848531797973,
  0.7887933577635216,
  0.475369416947535,
  0.22542996910336366,
  0.7036868092239419,
  0.3343328873486977,
  0.6374708949393826]}

In [17]:
res = bv_nairobi_results
for r in res:
    avg = np.mean(res[r])
    h_bound = np.max(res[r]) - avg
    l_bound = avg - np.min(res[r])
    print(r, " & ", "$", "%.2f" % avg, "\substack{+", "%.2f" % h_bound, " \\\\ -", "%.2f" %l_bound, "}$", sep='')

bare & $0.67\substack{+0.19 \\ -0.23}$
aim & $0.59\substack{+0.15 \\ -0.28}$
sim & $0.61\substack{+0.12 \\ -0.16}$
jigsaw & $0.45\substack{+0.33 \\ -0.34}$
cmc & $0.54\substack{+0.24 \\ -0.26}$
cmc_err & $0.58\substack{+0.21 \\ -0.26}$
m3 & $0.50\substack{+0.28 \\ -0.29}$


In [18]:
bv_quito_results = {'bare': [0.25956250000000003,
  0.265875,
  0.262625,
  0.26937500000000003,
  0.2646875,
  0.26387499999999997,
  0.2573125,
  0.249875,
  0.27362499999999995,
  0.2636875],
 'full': [0.029775313464926623,
  0.06647910744451674,
  0.04667775226332799,
  0.0332809168928796,
  0.05201336090306968,
  0.02816636396648242,
  0.042579175033529204,
  0.044728550884244545,
  0.04091080425915061,
  0.041340332580012795],
 'linear': [0.06685398871637849,
  0.10307654542937617,
  0.08341716484628997,
  0.0682755370919717,
  0.08884475703082517,
  0.061451295852230636,
  0.07950304239046785,
  0.08032007572549499,
  0.07567339834293518,
  0.07817281690779665],
 'aim': [0.27145833333333336,
  0.268125,
  0.26879166666666665,
  0.26529166666666665,
  0.26316666666666666,
  0.2535833333333333,
  0.25654166666666667,
  0.2699166666666667,
  0.26087499999999997,
  0.2625],
 'sim': [0.27584375,
  0.2750625,
  0.27134375,
  0.26821875,
  0.26899999999999996,
  0.26999999999999996,
  0.2675625,
  0.2629375,
  0.27165625,
  0.28025],
 'jigsaw': [0.20800916343216552,
  0.21512919397038882,
  0.25229220293853327,
  0.23102214725988712,
  0.24311394040792333,
  0.2042409427714018,
  0.166963203629511,
  0.16949084162224526,
  0.1937579000713494,
  0.22623957846867676],
 'cmc': [0.15388526001646624,
  0.16621548127143265,
  0.1570145400187437,
  0.16281618433360506,
  0.15862432406425914,
  0.1545232415778796,
  0.15268780422335748,
  0.14605907920957362,
  0.1701612808123521,
  0.15924838384240414], 
 'cmc_err':[0.2744379333219121,
  0.1695567970415347,
  0.16489292681487588,
  0.14460394095542106,
  0.15745310780670307,
  0.15711038837488828,
  0.1574260355967118,
  0.1669498632895472,
  0.17499063777335538]}

In [19]:
res = bv_quito_results
for r in res:
    avg = np.mean(res[r])
    h_bound = np.max(res[r]) - avg
    l_bound = avg - np.min(res[r])
    print(r, " & ", "$", "%.2f" % avg, "\substack{+", "%.2f" % h_bound, " \\\\ -", "%.2f" %l_bound, "}$", sep='')

bare & $0.26\substack{+0.01 \\ -0.01}$
full & $0.04\substack{+0.02 \\ -0.01}$
linear & $0.08\substack{+0.02 \\ -0.02}$
aim & $0.26\substack{+0.01 \\ -0.01}$
sim & $0.27\substack{+0.01 \\ -0.01}$
jigsaw & $0.21\substack{+0.04 \\ -0.04}$
cmc & $0.16\substack{+0.01 \\ -0.01}$
cmc_err & $0.17\substack{+0.10 \\ -0.03}$
